### Recap 07 (Ortschaften, Listcomprehension)
Uns interessiert, welche Schweizer Ortsnamen am häufigsten vorkommen.
Genauer: Welcher Ortsnamen bezeichnet am meisten verschiedene Orte.
Um diese Frage beantworten zu können, sollten wir festlegen, was ein Ortsname ist.
- Ist *Langnau* bereits ein Ortsname oder nur *Langnau im Emmental*?
- Ist der Ortsname *Bad Zurzach* oder nur *Zurzach*?
- Ist der Ortsname *Bad Ragaz* oder nur *Ragaz*?
- Ist der Ortsname *Sihlbrugg Station* oder *Shilbrugg*?

Die Ortsnamen in unserem Datensatz `L07/PLZO_CSV_LV95.txt` enthalten zudem oft noch
Gemeinde und Kanton, z.B. *Ottikon (Gossau ZH)*.

Wir verkürzen die Ortsnamen durch Weglassen beschreibenden Zusätze:
- Kantonskürzel und Angaben in Klammern (Gemeide, ...) lassen wir weg
- *im Emmental*, *an der Aare*, ... lassen wir weg
- Orte mit Ziffern lassen wir weg (*Lausanne 26*, *Root D4*) 


**Plan**:
- Zuerst erstellen wir eine sortierte Liste, die jeden Ortschaftsnamen nur einmal enthält und
  verschaffen uns einen Überblick über die Ortsnamen.
  Dazu benutzen wir fast ausschliesslich **Listkomprehension**.
- Mit Slice-Notation und der **String-Methode** `find` schneiden wir beschreibenden Zusätze weg.
- Dann arbeiten wir mit Dictionaries, um herauszufinden, welche Ortsnamen mit
  welchen Postleitzahlen zusammengehen.
  Wir finden z.B.  
    - `'Zürich': [8001, 8002, 8003, ..., 8057, 8064]`  
    - `'Rickenbach': [4462, 4613, 6221, 6432, 8545, 9532]`
 
  <br>  
  
  Im Falle von Zürich liegen die Postleitzahlen nahe beieinander (max. Differenz kleiner 100).
  Deshalb zählen wir Zürich als eine **Stadt**.
  Alle Postleitzahlen zu Rickenbach liegen dagegen weit auseinander (min. Differenz grösser 100).
  Alle diese Rickenbachs sind verschiedene **Dörfer**.

In [ ]:
filename = '../L07/PLZO_CSV_LV95.txt'
with open(filename, mode='r') as f:
    lines = [line.rstrip() for line in f]
lines[0]

In [ ]:
def format_row(row):
    return [row[0],  # Ortsname
            int(row[1]),  # PLZ
            row[3],  # Gemeinde
            row[5],  # Kantonskürzel
            (round(float(row[6])/1000-2600),  # km östlich von Bern Bhf
             round(float(row[7])/1000-1200),  # km nördlich von Bern Bhf
             ),
            ]

In [ ]:
# Erstelle Tabelle mit (Ortsname, PLZ, Gemeinde, Koords relativ zu Bhf Bern)
SEP = ';'
table = [line.split(SEP) for line in lines[1:]]
table = [format_row(row) for row in table]
table[:2] + table[-2:]

In [ ]:
# sortierte Liste mit den Ortsnamen (jeder Ortsname nur einmal, siehe Mengen.ipynb)
orte = sorted(list(set(row[0] for row in table)))
orte[:3] + ['...'] + orte[-3:]

In [ ]:
# Zeilen mit 'Aarau'
[row for row in table if 'Aarau' in row[0]]

In [ ]:
# Orte, die mit Abkuerzung beginnen
q = [ort for ort in orte if ort.split()[0].endswith('.')]
q[:2] + q[-2:]

In [ ]:
# Orte mit Abkuerzung nach dem ersten Space
q = [ort for ort in orte if ort.find(' ') < ort.find('.')]
q[:2] + q[-2:]

In [ ]:
# Orte mit Kantonskuerzel BE
q = [ort for ort in orte if 'BE' in ort]
q[:2] + q[-2:]

In [ ]:
# Orte mit '(' und Kantonskuerzel
q = [ort for ort in orte if '(' in ort and 'ZH' in ort]
q[:2] + q[-2:]

In [ ]:
# Orte mit beschreibenden Zusaetzen
ZUSAETZE = ('(', 'an', 'am', 'im', 'ob', 'in', 'b.', 'sopra',
            'Station', 'Dorf', 'Stadt', 'Platz',
            'Waldhaus', 'Hochwiese', 'Sulz', 'Waldegg',
            'Kulm', 'Hospiz',
            'Bad', 'See', 'Val ',
            'Herzogenbuchsee',
            )

for s in ZUSAETZE:
    print([ort for ort in orte if f' {s}' in ort][:2])

In [ ]:
[row for row in table if ' Station' in row[0]]

***
Ist der Test etwas komplizierter, schreiben wir eine Funktion dafür
***

In [ ]:
def contains_digits(s):
    for c in '01234567689':
        if c in s:
            return True


[ort for ort in orte if contains_digits(ort)]

In [ ]:
def contains_XX(s):
    '''gibt True zurueck, falls s
       zwei aufeinanderfolgende Grossbuchstaben enthaelt
    '''
    for x, y in zip(s, s[1:]):
        if x.isupper() and y.isupper():
            return True


q = [ort for ort in orte if contains_XX(ort)]
q[:2] + q[-2:]

In [ ]:
def find_XX(s):
    for i, (x, y) in enumerate(zip(s, s[1:])):
        if x.isupper() and y.isupper():
            return i
    return -1


def shorten(ort):
    i = find_XX(ort)  # Teil nach 'BE' entfernen
    if i != -1:
        ort = ort[:i-1]

    for sub in ZUSAETZE:  # Teil nach sub entfernen
        i = ort.find(' '+sub)
        if i != -1:
            ort = ort[:i]

    for sub in ('Appenzell', 'Aarau', 'Davos', 'Flumserberg', 'Hasliberg', 'Rigi'):
        i = ort.find(sub)
        if i != -1:
            ort = ort[:len(sub)]

    return ort

In [ ]:
for s in ('Ottikon (Gossau ZH)', 'Arni BE', 'Bueren and der Aare'):
    print(f'{s} -> {shorten(s)}')

In [ ]:
orte_short = set(shorten(ort) for ort in orte if not contains_digits(ort))
orte_short = sorted(orte_short)

In [ ]:
# Kurze Ortschaftsnamen mit mehr als einem SPACE
[ort for ort in orte_short if ort.count(' ') > 1]

In [ ]:
# Kurze Ortschaftsnamen mit einem SPACE (ohne Orte mit fuehrendem Artikel oder Abkz.)
q = [ort for ort in orte_short if ort.count(' ') == 1
     and '.' not in ort
     and not ort.startswith(('Les ', 'La ', 'Le '))]

[', '.join(q[i:i+3]) for i in range(0, len(q), 3)]

In [ ]:
[row for row in table if 'Rüschegg' in row[0]]

***
### 2.Teil (Dictionaries)
- Wir erstellen einen Menge mit `(ort, plz)` Paaren.  
- Wir erstellen einen Dict, der zu jedem Ort angibt, welche Postleitzahlen mit diesem Ort gepaart sind. 
  Zudem sortieren wir diese Postleitzahlenliste. Nachstehend zwei Key-Value Paare:  
  `('Luzern', [6003, 6004, 6005, 6006, 6014, 6015])`,    
  `('Sihlbrugg', [6340, 8135])`.  
  Liegen alle Postleitzahlen nahe beieinander und beginnen die Endziffern mit 00, ..., 04, so handelt es sich um eine Stadt, andernfalls ist es komplizierter.
  Wir behelfen uns mit der Entfernung:
  Z.B. liegen (Sihlbrugg Station, 8135) und (Sihlbrugg, 6340) weniger als 3km auseinander. Es handlet sich also um die gleiche Ortschaft.
***

In [ ]:
[row for row in table if row[0].startswith('Sihlbrugg')]

In [ ]:
from dict_tools import peek
from recap_07_II import show, distances

In [ ]:
orte_short_mit_plz = set((shorten(row[0]), row[1])
                         for row in table if not contains_digits(row[0])
                         )

In [ ]:
# 3 Elemente der Menge anzeigen
q = set()
for i, p in enumerate(orte_short_mit_plz):
    if i == 3:
        break
    q.add(p)
q

In [ ]:
ort_plzs = {}
for ort, plz in orte_short_mit_plz:
    # ist ort bereits key, so ist der zugehoerige Wert eine Liste
    # an die wir die plz anhaengen.
    # Anderfalls weisen wir dem Key ort die 1-elementige Liste [plz] zu
    if ort in ort_plzs:
        ort_plzs[ort].append(plz)
    else:
        ort_plzs[ort] = [plz]

ort_plzs = {k: sorted(v) for k, v in ort_plzs.items()}  # Postleitzahlen sortieren
# dict nach Laenge der Postleitzahlenliste sortieren
ort_plzs = dict(sorted(ort_plzs.items(), key=lambda x: len(x[1]), reverse=True))  # dict nach

peek(ort_plzs, n=1)

In [ ]:
def categorize(ort):
    plzs = ort_plzs[ort]
    if len(plzs) == 1:
        return 'single'
    elif plzs[0] % 100 <= 4 and plzs[0]//100 == plzs[-1]//100:
        return 'city'
    elif max(distances(ort)) <= 3:
        return 'same'
    elif min(distances(ort)) >= 9:
        return 'distinct'
    else:
        return '???'

In [ ]:
{k: v for k, v in ort_plzs.items() if categorize(k) == 'same'}

In [ ]:
q = {k: v for k, v in ort_plzs.items() if categorize(k) == '???'}
q

In [ ]:
for ort in q:
    show(ort)